In [1]:
import json
import os
import sys
import numpy as np
from gensim.models import Word2Vec
from keras.preprocessing import sequence
from keras.layers import merge, Dense, Input,Dropout, Embedding, LSTM, Bidirectional, Activation
from keras.layers import Conv2D,Conv1D
from keras.layers.merge import dot, multiply, add, concatenate
from keras.layers import Merge
from keras.layers.core import Lambda,Reshape, Flatten
from keras.layers.pooling import GlobalMaxPooling2D,GlobalMaxPooling1D
from keras.models import Model
from keras.backend import transpose,batch_dot,expand_dims
from keras import optimizers
from HomeDepotCSVReader import HomeDepotReader
import Utilities
from DataPreprocessing import DataPreprocessing
from Feature_Word2Vec import Feature_Word2Vec
from AutomaticQueryExpansion import Word2VecQueryExpansion
import re
from nltk.corpus import stopwords
from keras.utils.np_utils import to_categorical
import pandas as pd
from FeatureEngineering import HomeDepotFeature
from keras.layers.wrappers import TimeDistributed
from keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.metrics import mean_squared_error
from Evaluation import NDCG_Eval

Using TensorFlow backend.


In [36]:
# train_filename = '../data/train_play.csv'
# test_filename = '../data/test_play.csv'
# attribute_filename = '../data/attributes_play.csv'
# description_filename = '../data/product_descriptions_play.csv'
# word2vec_model_path='model/word2vec_play.model'
# vocab_path='model/word2vec_play_vocab.json'
# embeddings_path='model/embeddings_play.npz'

train_filename = '../data/train.csv'
test_filename = '../data/test.csv'
soln_filename = '../data/solution.csv'
attribute_filename = '../data/attributes.csv'
description_filename = '../data/product_descriptions.csv'
word2vec_model_path='model/word2vec.model'
vocab_path='model/word2vec_vocab.json'
embeddings_path='model/embeddings.npz'
#full_features_filename = '../data/features_full_plusnouns_pluspuidthreshpluss2vsimscore.csv'
full_features_filename = '../data/features_doc2vec_sense2vec_20170416.csv'
#dnn_pred_filename='../data/dnn_private.csv'
dnn_private_pred_filename='../data/dnn_private_256_0.2drop_colorincl_run_20160418-2.csv'
dnn_public_pred_filename='../data/dnn_public_256_0.2drop_colorincl_run_20160418-2.csv'
xgboost_private_pred_filename='../data/xgboost_private.csv'
xgboost_public_pred_filename='../data/xgboost_public.csv'
or_private_pred_filename='../data/ordinal_private.csv'
or_public_pred_filename='../data/ordinal_public.csv'


In [37]:
# train_query_df, product_df, attribute_df, test_query_df = reader.getQueryProductAttributeDataFrame(train_filename,
#                                               test_filename,
#                                               attribute_filename,
#                                               description_filename)
# print("train_query_df:",list(train_query_df))
# print("product_df:", list(product_df))
# print("attribute_df:", list(attribute_df))
# print("test_query_df:", list(test_query_df))

In [38]:
soln_df = pd.read_csv(soln_filename, delimiter=',', low_memory=False, encoding="ISO-8859-1")
print(soln_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166693 entries, 0 to 166692
Data columns (total 3 columns):
id           166693 non-null int64
relevance    166693 non-null float64
Usage        166693 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 3.8+ MB
None


In [39]:
reader = HomeDepotReader()
feature_df = reader.getBasicDataFrame(full_features_filename)

In [40]:
dp = DataPreprocessing()
feature_test_df=feature_df.copy()
feature_test_df.pop('relevance')
test_private_df = dp.getGoldTestSet(feature_test_df, soln_df,
                                    testsetoption='Private')  # ,savepath='../data/test_private_gold.csv')
test_public_df = dp.getGoldTestSet(feature_test_df, soln_df,
                                    testsetoption='Public')  # ,savepath='../data/test_private_gold.csv')

In [41]:
dnn_public_df = reader.getBasicDataFrame(dnn_public_pred_filename)
xgboost_public_df = reader.getBasicDataFrame(xgboost_public_pred_filename)
or_public_df = reader.getBasicDataFrame(or_public_pred_filename)
dnn_private_df = reader.getBasicDataFrame(dnn_private_pred_filename)
xgboost_private_df = reader.getBasicDataFrame(xgboost_private_pred_filename)
or_private_df = reader.getBasicDataFrame(or_private_pred_filename)

In [42]:
#test_private_df

In [43]:
RMSE = mean_squared_error(test_public_df.relevance.as_matrix(), xgboost_public_df.relevance)**0.5
print(RMSE)

0.468870503972


In [44]:
RMSE = mean_squared_error(test_public_df.relevance.as_matrix(), dnn_public_df.pred_relevance)**0.5
print(RMSE)

0.472106575935


In [45]:
RMSE = mean_squared_error(test_public_df.relevance.as_matrix(), or_public_df.pred_relevance)**0.5
print(RMSE)

0.482698829879


In [46]:
public_combined_avg=(xgboost_public_df.relevance*0.75+dnn_public_df.pred_relevance*0.25)#+or_df.pred_relevance*0.1)

In [47]:
RMSE = mean_squared_error(test_public_df.relevance.as_matrix(), public_combined_avg)**0.5
print(RMSE)

0.468266214064


# private

In [48]:
RMSE = mean_squared_error(test_private_df.relevance.as_matrix(), xgboost_private_df.relevance)**0.5
print(RMSE)

0.465506528452


In [49]:
RMSE = mean_squared_error(test_private_df.relevance.as_matrix(), dnn_private_df.pred_relevance)**0.5
print(RMSE)

0.469481831722


In [50]:
RMSE = mean_squared_error(test_private_df.relevance.as_matrix(), or_private_df.pred_relevance)**0.5
print(RMSE)

0.480283051052


In [51]:
private_combined_avg=(xgboost_private_df.relevance*0.75+dnn_private_df.pred_relevance*0.25)#+or_df.pred_relevance*0.1)

In [52]:
RMSE = mean_squared_error(test_private_df.relevance.as_matrix(), private_combined_avg)**0.5
print(RMSE)

0.465069184523


# save results

In [53]:
dnn_test_public_pred_df = pd.DataFrame(test_public_df[['id','search_term','product_uid']])
dnn_test_public_pred_df.reset_index(drop=True, inplace=True)
dnn_test_public_prediction_ori_scale_df = pd.DataFrame(public_combined_avg,columns=['relevance_int']) # to fit in interface to ncdg
dnn_test_public_prediction_ori_scale_df
dnn_test_public_pred_df=dnn_test_public_pred_df.join(dnn_test_public_prediction_ori_scale_df)

In [54]:
dnn_test_private_pred_df = pd.DataFrame(test_private_df[['id','search_term','product_uid']])
dnn_test_private_pred_df.reset_index(drop=True, inplace=True)
dnn_test_private_prediction_ori_scale_df = pd.DataFrame(private_combined_avg,columns=['relevance_int']) # to fit in interface to ncdg
dnn_test_private_prediction_ori_scale_df
dnn_test_private_pred_df=dnn_test_private_pred_df.join(dnn_test_private_prediction_ori_scale_df)

In [55]:
ensemble_test_public_pred_df = pd.DataFrame(test_public_df[['id','search_term','product_uid']])
ensemble_test_public_pred_df.reset_index(drop=True, inplace=True)
ensemble_test_public_prediction_ori_scale_df = pd.DataFrame(public_combined_avg,columns=['relevance_int']) # to fit in interface to ncdg
ensemble_test_public_prediction_ori_scale_df
ensemble_test_public_pred_df=ensemble_test_public_pred_df.join(ensemble_test_public_prediction_ori_scale_df)

In [56]:
ensemble_test_private_pred_df = pd.DataFrame(test_private_df[['id','search_term','product_uid']])
ensemble_test_private_pred_df.reset_index(drop=True, inplace=True)
ensemble_test_private_prediction_ori_scale_df = pd.DataFrame(private_combined_avg,columns=['relevance_int']) # to fit in interface to ncdg
ensemble_test_private_prediction_ori_scale_df
ensemble_test_private_pred_df=ensemble_test_private_pred_df.join(ensemble_test_private_prediction_ori_scale_df)

In [57]:
ensemble_test_private_pred_df

,id,search_term,product_uid,relevance_int
0,5,simpson sku abl,100001,2.177266
1,6,simpson strong tie,100001,2.770995
2,10,bath shower kit,100003,2.578766
3,15,delta ashland shower faucet,100005,2.302421
4,24,self tap screw,100008,2.464913
5,26,door trim,100009,2.190027
6,28,anchor stake,100010,2.522550
7,30,lawn edg,100010,2.199106
8,33,steel landscap edg,100010,2.212571
9,39,honda push mower,100011,2.495360


In [58]:
# pd.DataFrame(ensemble_test_private_pred_df).to_csv('../data/ensemble_private_xgb_0_75_dnn_new_0_25.csv', \
#                                     index=False, header=True)

In [59]:
pd.DataFrame(ensemble_test_private_pred_df).to_csv('../data/ensemble_private.csv', \
                                    index=False, header=True)

In [60]:
pd.DataFrame(ensemble_test_public_pred_df).to_csv('../data/ensemble_public.csv', \
                                    index=False, header=True)

# ncdg

In [61]:
ndcgEval=NDCG_Eval()

In [62]:
test_private_df.drop('relevance_int', axis=1, inplace=True)
test_private_df.rename(columns={'relevance': 'relevance_int'}, inplace=True) #for ncdg

In [63]:
test_private_df

,id,product_idx,product_uid,search_term,color_exist,color1hot_almond,color1hot_aluminum,color1hot_beige,color1hot_biscuit,color1hot_bisque,...,len_product_title,len_product_description,len_brand,len_search_term,sense2vec_all_simscore,sense2vec_keeptag_simscore,sense2vec_uidfact_all_simscore,sense2vec_uidfact_keeptag_simscore,product_uid_threshold,relevance_int
74069,5,"Int64Index([0], dtype='int64')",100001,simpson sku abl,0,0,0,0,0,0,...,6,102,3,3,0.111254,0.062794,0.111254,0.062794,1,2.33
74070,6,"Int64Index([0], dtype='int64')",100001,simpson strong tie,0,0,0,0,0,0,...,6,102,3,3,0.093536,0.093076,0.093536,0.093076,1,2.67
74073,10,"Int64Index([54667], dtype='int64')",100003,bath shower kit,0,0,0,0,0,0,...,21,79,1,3,0.327519,0.122193,0.327519,0.122193,1,2.67
74078,15,"Int64Index([2], dtype='int64')",100005,delta ashland shower faucet,0,0,0,0,0,0,...,11,78,1,4,-0.007135,-0.010146,-0.007135,-0.010146,1,2.67
74081,24,"Int64Index([54669], dtype='int64')",100008,self tap screw,0,0,0,0,0,0,...,16,78,1,3,0.372281,0.206755,0.372281,0.206755,1,3.00
74083,26,"Int64Index([5], dtype='int64')",100009,door trim,0,0,0,0,0,0,...,14,60,3,2,0.338932,0.115508,0.338932,0.115508,1,2.00
74084,28,"Int64Index([6], dtype='int64')",100010,anchor stake,0,0,0,0,0,0,...,7,81,3,2,0.342431,0.153442,0.342431,0.153442,1,2.00
74086,30,"Int64Index([6], dtype='int64')",100010,lawn edg,0,0,0,0,0,0,...,7,81,3,2,0.341220,0.122325,0.341220,0.122325,1,1.00
74089,33,"Int64Index([6], dtype='int64')",100010,steel landscap edg,1,0,0,0,0,0,...,7,81,3,3,0.361824,0.146137,0.361824,0.146137,1,2.00
74091,39,"Int64Index([7], dtype='int64')",100011,honda push mower,0,0,0,0,0,0,...,17,188,1,3,-0.005698,0.014031,-0.005698,0.014031,1,2.67


In [64]:
test_public_df.drop('relevance_int', axis=1, inplace=True)
test_public_df.rename(columns={'relevance': 'relevance_int'}, inplace=True) #for ncdg

In [65]:
test_public_df

,id,product_idx,product_uid,search_term,color_exist,color1hot_almond,color1hot_aluminum,color1hot_beige,color1hot_biscuit,color1hot_bisque,...,len_product_title,len_product_description,len_brand,len_search_term,sense2vec_all_simscore,sense2vec_keeptag_simscore,sense2vec_uidfact_all_simscore,sense2vec_uidfact_keeptag_simscore,product_uid_threshold,relevance_int
74068,4,"Int64Index([0], dtype='int64')",100001,metal l bracket,0,0,0,0,0,0,...,6,102,3,3,-0.013432,-0.007869,-0.013432,-0.007869,1,2.33
74071,7,"Int64Index([0], dtype='int64')",100001,simpson strong tie hcc668,0,0,0,0,0,0,...,6,102,3,4,0.101048,0.077886,0.101048,0.077886,1,2.00
74075,12,"Int64Index([54667], dtype='int64')",100003,one piec tub shower,0,0,0,0,0,0,...,21,79,1,4,0.294785,0.119400,0.294785,0.119400,1,2.00
74076,13,"Int64Index([54668], dtype='int64')",100004,solar panel,0,0,0,0,0,0,...,9,102,2,2,0.268002,0.163241,0.268002,0.163241,1,3.00
74077,14,"Int64Index([2], dtype='int64')",100005,1 handl shower delta trim kit,0,0,0,0,0,0,...,11,78,1,6,0.297397,0.108830,0.297397,0.108830,1,2.00
74082,25,"Int64Index([5], dtype='int64')",100009,3 1 20door case,0,0,0,0,0,0,...,14,60,3,4,0.217235,0.102798,0.217235,0.102798,1,2.33
74092,40,"Int64Index([7], dtype='int64')",100011,lawn mower,0,0,0,0,0,0,...,17,188,1,2,0.388901,0.226367,0.388901,0.226367,1,3.00
74097,45,"Int64Index([7], dtype='int64')",100011,toro lawn mower bag,0,0,0,0,0,0,...,17,188,1,4,0.023385,0.001151,0.023385,0.001151,1,2.67
74098,46,"Int64Index([7], dtype='int64')",100011,toro mower washout,0,0,0,0,0,0,...,17,188,1,3,0.014796,0.001652,0.014796,0.001652,1,2.67
74102,52,"Int64Index([9], dtype='int64')",100013,garbag dispos,0,0,0,0,0,0,...,8,67,1,2,0.035256,0.020145,0.035256,0.020145,1,2.00


In [66]:
avgNDCG=ndcgEval.computeAvgNDCG(test_public_df,dnn_test_public_pred_df)

Filtering essential columns
goldDF columns: ['relevance_int', 'search_term', 'product_uid']
predictDF columns: ['relevance_int', 'search_term', 'product_uid']
Completed: Filtering essential columns
STARTING AVG_NDCG computation...this operation can take a while..
Sorting by query small to big, relevance big to small
Completed: Sorting by query small to big, relevance big to small
Sorting predictdf according to prediction relevance
Completed: Sorting predictdf according to prediction relevance
Attaching the relevance_score computed in PreTrain to each query-doc in the query-docs set
Completed: Attaching the relevance_score computed in PreTrain to each query-doc in the query-docs set
Computing DCGp of predicted sets
Compeleted: Computing DCGp of predicted sets
Ensuring data correctness before final computation
No of search terms removed from NDCG:  0 []
Completed: Ensuring data correctness before final computation
Computing NDCG = DCGp / DCGmax
Completed: Computing NDCG = DCGp / DCGmax
C

In [67]:
avgNDCG=ndcgEval.computeAvgNDCG(test_private_df,dnn_test_private_pred_df)

Filtering essential columns
goldDF columns: ['relevance_int', 'search_term', 'product_uid']
predictDF columns: ['relevance_int', 'search_term', 'product_uid']
Completed: Filtering essential columns
STARTING AVG_NDCG computation...this operation can take a while..
Sorting by query small to big, relevance big to small
Completed: Sorting by query small to big, relevance big to small
Sorting predictdf according to prediction relevance
Completed: Sorting predictdf according to prediction relevance
Attaching the relevance_score computed in PreTrain to each query-doc in the query-docs set
Completed: Attaching the relevance_score computed in PreTrain to each query-doc in the query-docs set
Computing DCGp of predicted sets
Compeleted: Computing DCGp of predicted sets
Ensuring data correctness before final computation
No of search terms removed from NDCG:  0 []
Completed: Ensuring data correctness before final computation
Computing NDCG = DCGp / DCGmax
Completed: Computing NDCG = DCGp / DCGmax
C

In [68]:
avgNDCG=ndcgEval.computeAvgNDCG(test_private_df,ensemble_test_private_pred_df)

Filtering essential columns
goldDF columns: ['relevance_int', 'search_term', 'product_uid']
predictDF columns: ['relevance_int', 'search_term', 'product_uid']
Completed: Filtering essential columns
STARTING AVG_NDCG computation...this operation can take a while..
Sorting by query small to big, relevance big to small
Completed: Sorting by query small to big, relevance big to small
Sorting predictdf according to prediction relevance
Completed: Sorting predictdf according to prediction relevance
Attaching the relevance_score computed in PreTrain to each query-doc in the query-docs set
Completed: Attaching the relevance_score computed in PreTrain to each query-doc in the query-docs set
Computing DCGp of predicted sets
Compeleted: Computing DCGp of predicted sets
Ensuring data correctness before final computation
No of search terms removed from NDCG:  0 []
Completed: Ensuring data correctness before final computation
Computing NDCG = DCGp / DCGmax
Completed: Computing NDCG = DCGp / DCGmax
C

In [69]:
avgNDCG=ndcgEval.computeAvgNDCG(test_public_df,ensemble_test_public_pred_df)

Filtering essential columns
goldDF columns: ['relevance_int', 'search_term', 'product_uid']
predictDF columns: ['relevance_int', 'search_term', 'product_uid']
Completed: Filtering essential columns
STARTING AVG_NDCG computation...this operation can take a while..
Sorting by query small to big, relevance big to small
Completed: Sorting by query small to big, relevance big to small
Sorting predictdf according to prediction relevance
Completed: Sorting predictdf according to prediction relevance
Attaching the relevance_score computed in PreTrain to each query-doc in the query-docs set
Completed: Attaching the relevance_score computed in PreTrain to each query-doc in the query-docs set
Computing DCGp of predicted sets
Compeleted: Computing DCGp of predicted sets
Ensuring data correctness before final computation
No of search terms removed from NDCG:  0 []
Completed: Ensuring data correctness before final computation
Computing NDCG = DCGp / DCGmax
Completed: Computing NDCG = DCGp / DCGmax
C

# correlation

In [ ]:
print(test_private_df.relevance.corr(test_private_df.relevance,method='pearson'))
print(test_private_df.relevance.corr(xgboost_df.relevance,method='pearson'))
print(test_private_df.relevance.corr(dnn_df.pred_relevance,method='pearson'))
print(test_private_df.relevance.corr(or_df.pred_relevance,method='pearson'))

In [ ]:
print(xgboost_df.relevance.corr(dnn_df.pred_relevance,method='pearson'))
print(xgboost_df.relevance.corr(or_df.pred_relevance,method='pearson'))

In [ ]:
print(dnn_df.pred_relevance.corr(or_df.pred_relevance,method='pearson'))